Las herramientas y recursos pueden no estar ubicados en tu máquina local, pueden estar en cualquier lugar de Internet; solo necesitamos una forma estándar de comunicarnos con esos puntos de entrada.

# Protocolo de Contexto de Modelo

El Protocolo de Contexto de Modelo (MCP, por sus siglas en inglés) es un estándar abierto diseñado para facilitar la comunicación fluida entre modelos de aprendizaje automático, herramientas y recursos, sin importar su ubicación física. MCP define un conjunto de convenciones y APIs que permiten la interoperabilidad, haciendo más fácil integrar modelos y servicios a través de diversas plataformas y entornos.

Las características clave de MCP incluyen:
- **APIs estandarizadas** para la invocación de modelos e intercambio de contexto
- **Soporte para recursos remotos y distribuidos**
- **Extensibilidad** para acomodar varios tipos de modelos y flujos de trabajo

Para información más detallada, consulta la [Especificación del Protocolo de Contexto de Modelo](https://github.com/modelcontext/protocol) y la [documentación oficial](https://modelcontext.org/docs/protocol).

In [ ]:
%%writefile my_server.py
from random import random
from fastmcp import FastMCP

# Instrucciones sobre como actuar
mcp = FastMCP(
    name="Asistente",
    instructions="""
        Eres un asistente de ayuda.
        La función multiply() permite multiplicar dos valores.
    """,
)

@mcp.tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

@mcp.resource("data://config")
def config() -> dict:
    """MCP server configuration"""
    return {"verbose" : True, "seed" : 1234}

@mcp.prompt
def simple_prompt(data_points: list[float]) -> str:
    """Creates a prompt asking help on some data points"""
    formatted_data = ", ".join(str(point) for point in data_points)
    return f"Please analyze these data points: {formatted_data}"

# Run the server
if __name__ == "__main__":
    mcp.run()

Overwriting my_server.py


Ahora, puedes ejecutar el servidor en un terminal.

```
fastmcp run my_server.py:mcp
```

con el entornos activado o poniendo delante `uv run` para permitir que uv gestione las dependencias.

## Herramientas

In [35]:
from fastmcp import Client

client = Client("my_server.py")

async with client:
    tools = await client.list_tools()
    
    for tool in tools:
        print(f"Tool: {tool.name}")
        print(f"Description: {tool.description}")
        if tool.inputSchema:
            print(f"Parameters: {tool.inputSchema}")

Tool: multiply
Description: Multiplies two numbers together.
Parameters: {'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'type': 'object'}


## Recursos

In [36]:
async with client:
    resources = await client.list_resources()
    
    for resource in resources:
        print(f"Resource URI: {resource.uri}")
        print(f"Name: {resource.name}")
        print(f"Description: {resource.description}")
        print(f"MIME Type: {resource.mimeType}")

Resource URI: data://config
Name: config
Description: MCP server configuration
MIME Type: text/plain


## Prompts

In [37]:
async with client:
    prompts = await client.list_prompts()
    
    for prompt in prompts:
        print(f"Prompt: {prompt.name}")
        print(f"Description: {prompt.description}")
        if prompt.arguments:
            print(f"Arguments: {[arg.name for arg in prompt.arguments]}")

Prompt: simple_prompt
Description: Creates a prompt asking help on some data points
Arguments: ['data_points']


## LLamadas

In [38]:
async with client:
    result = await client.call_tool("multiply", {"a": 4, "b" : 10})
    print(result)

CallToolResult(content=[TextContent(type='text', text='40.0', annotations=None, meta=None)], structured_content={'result': 40.0}, data=40.0, is_error=False)


In [39]:
async with client:
    result = await client.get_prompt("simple_prompt", {"data_points": [1, 2, 3]})
    print(result)

meta=None description='Creates a prompt asking help on some data points' messages=[PromptMessage(role='user', content=TextContent(type='text', text='Please analyze these data points: 1.0, 2.0, 3.0', annotations=None, meta=None))]


In [40]:
async with client:
    number = await client.read_resource("data://config")
    print(number)

[TextResourceContents(uri=AnyUrl('data://config'), mimeType='text/plain', meta=None, text='{"verbose":true,"seed":1234}')]


## Servidores MCP externos

Existen muchas aplicaciones que ofrecen servidores MCP externos, además de la gran cantidad de servidores API disponibles para casi todas las soluciones de productividad.

https://mcpservers.org/remote-mcp-servers

Puedes probar con el servidor MCP de Github, por ejemplo https://api.githubcopilot.com/mcp/ o crear un MCP desde cualquier repositorio con https://gitmcp.io/ o una instancia de Github Pages. Por ejemplo, tomemos la documentación de LangChain.

https://langchain-ai.github.io/langchain

In [51]:
from fastmcp import Client

# We need to change github for gitmcp
client = Client(f"https://langchain-ai.gitmcp.io/langchain")

async with client:
    tools = await client.list_tools()
    
    for tool in tools:
        print(f"Tool: {tool.name}")
        print(f"Description: {tool.description}")
        if tool.inputSchema:
            print(f"Parameters: {tool.inputSchema}")

Unknown SSE event: endpoint


Tool: fetch_langchain_documentation
Description: Fetch entire documentation file from the langchain-ai/langchain GitHub Pages. Useful for general questions. Always call this tool first if asked about langchain-ai/langchain.
Parameters: {'type': 'object'}
Tool: search_langchain_documentation
Description: Semantically search within the fetched documentation from the langchain-ai/langchain GitHub Pages. Useful for specific queries.
Parameters: {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query to find relevant documentation'}}, 'required': ['query'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}
Tool: search_langchain_code
Description: Search for code within the GitHub repository: "langchain-ai/langchain" using the GitHub Search API (exact match). Returns matching files for you to query further if relevant.
Parameters: {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search 

In [56]:
import pprint

async with client:
    result = await client.call_tool("search_langchain_documentation", {"query" : "memory"})
    pprint.pprint(result.content[0])

Unknown SSE event: endpoint


TextContent(type='text', text='## Query\n\nmemory.\n\n## Response\n\n### Sources:\nImportant: you can fetch the full content of any source using the fetch_url_content tool\n\n#### (langchain-ai/langchain/docs/docs/versions/migrating_memory/index.mdx)[https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/versions/migrating_memory/index.mdx] (Score: 0.54)\n- \n\nPlease see [long-term memory agent tutorial](long_term_memory_agent) implements an agent that can extract structured information from the conversation history.\n\nMemory classes that fall into this category include:\n\n| Memory Type                | Description                                                                                                                                                                                                       |\n|----------------------------|----------------------------------------------------------------------------------------------------------------------------

De manera similar, puedes añadir esta conexión a tu asistente de codificación para que pueda realizar las mismas acciones (consultar herramientas, recursos o lo que exponga el servidor MCP).

```
{
  "servers": {
    "langchain Docs": {
      "type": "sse",
      "url": "https://langchain-ai.gitmcp.io/langchain"
    }
  }
}
```